# Question 2: Gradient Descent Boosting: Hastie et al. algorithm 10.4

In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [18]:
data = pd.read_csv("Heart.csv")
data.dropna(inplace=True)
data.drop(columns="Unnamed: 0",inplace=True)
print(data.head(5))

   Age  Sex     ChestPain  RestBP  Chol  Fbs  RestECG  MaxHR  ExAng  Oldpeak  \
0   63    1       typical     145   233    1        2    150      0      2.3   
1   67    1  asymptomatic     160   286    0        2    108      1      1.5   
2   67    1  asymptomatic     120   229    0        2    129      1      2.6   
3   37    1    nonanginal     130   250    0        0    187      0      3.5   
4   41    0    nontypical     130   204    0        2    172      0      1.4   

   Slope   Ca        Thal  AHD  
0      3  0.0       fixed   No  
1      2  3.0      normal  Yes  
2      2  2.0  reversable  Yes  
3      3  0.0      normal   No  
4      1  0.0      normal   No  


In [19]:
a = np.asarray(data)
unique, counts = np.unique(data["Thal"], return_counts=True)
print(np.asarray((unique, counts)))

[['fixed' 'normal' 'reversable']
 [18 164 115]]


In [20]:
from keras.utils import to_categorical

X = data.iloc[:,data.columns!="AHD"]
X = X.iloc[:,X.columns!="Thal"]
Y  = data.iloc[:,-2:]

dictionary = {"nonanginal":0,"asymptomatic":1,"typical":2,"nontypical":3}
X = X.replace(dictionary)
label=[]

for i,j in zip(Y["AHD"].values,Y["Thal"].values):
    if i == "No" and j == "fixed":
        label.append(0)
    elif i == "No" and  j == "normal":
        label.append(1)
    elif i == "No" and  j == "reversable":
        label.append(2)
    elif i == "Yes" and j == "fixed":
        label.append(3)
    elif i == "Yes" and j == "normal":
        label.append(4)
    elif i == "Yes" and j == "reversable":
        label.append(5)

label = pd.DataFrame(label,columns=["label"])

Y1 = to_categorical(label,6)
Y1=Y1.astype(np.uint8)
Y1 = pd.DataFrame(Y1,columns=["N_fixed","N_normal","N_reversable","Y_fixed","Y_normal","Y_reversable"])

Y.drop(columns=['AHD', 'Thal'])
Y = pd.concat([label,Y1], axis=1)

NameError: name 'to_categorical' is not defined

(a) Please implement algorithm 10.4 for multi-class gradient boosting. You must manually implement this. If you use other sources for coding help you must cite them.

In [21]:
import math
from sklearn.tree import DecisionTreeRegressor

class Gradient_Boosting:
    
    n_tree=None
    depth= None
    tree_gamma_dictionary ={}
    tree_dictionary={}
    F_train_dict={}
    F_test_dict={}
    fearure_importance ={}
    
        
    def __init__(self,n_tree,depth):
        self.n_tree = n_tree
        self.depth = depth
        self.tree_gamma_dictionary ={}
        self.tree_dictionary={}
        self.F_train_dict={}
        self.F_test_dict={}

    def cal_feature_importance(self):
        feature_importance = [self.tree_dictionary[tree].feature_importances_ for tree in self.tree_dictionary]
        fm=np.mean(feature_importance,axis=0)
        return fm
        
    def F_1(self,X,s):
        if s=="fit":
            if len(self.tree_dictionary)==0:
                a= np.zeros((len(X),6)).astype(np.uint8)
                self.F_train_dict[0]=a
                return a
            else:
                i = len(self.tree_dictionary)
                print(i)
                b=self.F_train_dict[i-1]
                a=[]
                regions=self.tree_dictionary[i-1].apply(X)
                regions = np.array(regions)
                for j in regions:
                    gamma = self.tree_gamma_dictionary[i-1][j]
                    a.append(gamma)
                a=np.array(a)
                b=np.add(a,b)
                self.F_train_dict[i]=b

                return b

        elif s=="predict":
            if len(self.tree_dictionary)==0:
                a= np.zeros((len(X),6)).astype(np.uint8)
                self.F_test_dict[0]=a
                return a
            else:
                b=np.zeros((len(X),6))
                for i in self.tree_dictionary:
                    a=[]
                    regions=self.tree_dictionary[i].apply(X)
                    for j in regions:
                        gamma = self.tree_gamma_dictionary[i][j]
                        a.append(gamma)
                    b=np.add(a,b)

                return b

            
    def F(self,X):
        if len(self.tree_dictionary)==0:
            a= np.zeros((len(X),6)).astype(np.uint8)
            return a
        else:
            b=np.zeros((len(X),6))
            for i in self.tree_dictionary:
                a=[]
                regions=self.tree_dictionary[i].apply(X)
                for j in regions:
                    gamma = self.tree_gamma_dictionary[i][j]
                    a.append(gamma)
                b=np.add(a,b)
                

            return b

    def predict(self,X):

        F_val = self.F(X)
        predictions = self.Pk(F_val)

        output = []
        for i in predictions:
            temp =[]
            label = np.argmax(i)
            for j in range(len(i)):
                if j==label:
                    temp.append(1)
                else:
                    temp.append(0)      
            output.append(temp)
        return output
    
    def Pk(self,F_val):
        output=[]
        for row in F_val:
            b=np.sum(np.exp(row))
            a=np.exp(row)
            d = a/b 
            output.append(d)
        return output
    
    def fit(self,X,Y):
        
        f_importance=np.zeros((len(X),6))
        
        for i in range(self.n_tree):

            numerator_dict ={}
            denominator_dict={}

            F_val = self.F(X)
            predictions = self.Pk(F_val)

            
            residuals = np.asarray(np.array(Y)-predictions)

            regressor = DecisionTreeRegressor(random_state=0,max_depth=self.depth)
            regressor.fit(X,residuals)
            
            X_regions = regressor.apply(X)

            for j in range(len(X_regions)):
                if X_regions[j] not in numerator_dict:
                    numerator_dict[X_regions[j]]=residuals[j]
                    denominator_dict[X_regions[j]]=abs(residuals[j])*(1-abs(residuals[j]))
                else:
                    numerator_dict[X_regions[j]]=np.add(numerator_dict[X_regions[j]],residuals[j])
                    denominator_dict[X_regions[j]]=np.add(denominator_dict[X_regions[j]],abs(residuals[j])*(1-abs(residuals[j])))
                    
            temp_dict={}
            for k in numerator_dict:
                temp = numerator_dict[k]/denominator_dict[k]
                temp_dict[k]=temp/6
            
            self.tree_gamma_dictionary[i] = temp_dict
            self.tree_dictionary[i]=regressor

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

tree_list = [50,100,150,200,250,300,350,400,500,550,600,700,800]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y.iloc[:,1:].values,random_state=40,test_size=0.2,stratify=Y.iloc[:,1:].values)

GB_loss_dict = {}

for i in tree_list:
    print(i)
    a = Gradient_Boosting(i,3)

    a.fit(X_train,Y_train)
     
    Y_pred = a.predict(X_test)
    
    Loss = log_loss(Y_test,Y_pred)
    print(Loss)
    GB_loss_dict[i]= Loss
    

    

50


TypeError: unsupported operand type(s) for -: 'str' and 'float'

(b) Comparison of your implementation with a popular package (XGBoost, GBM, or Light
GBM - your choice): Using the Heart dataset - develop a multi-class classification for Heart
Disease (AHD) and Thal crossed together (this requires you to create labels based upon the
combination of Yes/NO and Fixed, Normal, Reversable for the final two columns). Investigate
the number of iterations needed to optimize the testing error, then make the testing error start
to rise for your implementation and the package you choose to compare against. For multiclass
classification - please determine what metric you will be optimizing for comparison (note that
AUROC only works in binary classification).

In [23]:
from xgboost import XGBClassifier

X_train, X_test, Y_train, Y_test = train_test_split(X, Y.iloc[:,0].values,random_state=40, test_size=0.2,stratify=Y.iloc[:,0].values)
s = [50,100,150,200,250,300,350,400,500,550,600,700,800]
loss_dict = {}
for i in s:
    xgb_c = XGBClassifier(max_depth=3, n_estimators=i, n_jobs=-1)
    xgb_c.fit(X_train, Y_train,)
    pred   = xgb_c.predict(X_test)

    pred = to_categorical(pred,6)
    Y_test=Y_test.astype(np.uint8)
    
    Loss = log_loss(Y_test,pred)
    loss_dict[i]= Loss


plt.figure(figsize = (16,10))
plt.scatter([tree for tree in s], [loss_dict[tree] for tree in s])
plt.scatter([tree for tree in tree_list], [GB_loss_dict[tree] for tree in tree_list],color="red")
plt.plot([tree for tree in s], [loss_dict[tree] for tree in s])
plt.plot([tree for tree in tree_list], [GB_loss_dict[tree] for tree in tree_list],color="red")
plt.title('Loss for XGBOOST Package')
plt.xlabel('Trees')
plt.ylabel('Loss')
plt.show()

ModuleNotFoundError: No module named 'xgboost'

(c) Please provide your feature importance, and compare to the package choice’s feature importance. Describe similarity vs. differences and explain (in particular with relation to hyperparameters).


In [24]:
asd=a.cal_feature_importance()
e=pd.DataFrame(asd,columns=["Feature Importance"])
f=pd.DataFrame(X_train.columns,columns=["Feature Nmaes"])
g=pd.concat([f,e],axis=1)
g.sort_values(by=['Feature Importance'])

print("Feature Importance of our model")

print(g)

xgb_c = XGBClassifier(max_depth=3, n_estimators=800, n_jobs=-1)
xgb_c.fit(X_train, Y_train,)
asd = xgb_c.feature_importances_
    
e=pd.DataFrame(asd,columns=["Feature Importance"])
f=pd.DataFrame(X_train.columns,columns=["Feature Nmaes"])

g=pd.concat([f,e],axis=1)
g.sort_values(by=['Feature Importance'])

print("XGBoost Feature Importance")

print(g.head(40))


ValueError: DataFrame constructor not properly called!

In [25]:
import math
from sklearn.tree import DecisionTreeRegressor

class Gradient_Boosting:
    
    n_tree=None
    depth= None
    tree_gamma_dictionary ={}
    tree_dictionary={}
    F_dict={}
        
    def __init__(self,n_tree,depth):
        self.n_tree = n_tree
        self.depth = depth
        
    def F(self,X):
        if len(self.tree_dictionary)==0:
            a= np.zeros((len(X),6)).astype(np.uint8)
            return a
        else:
            b=np.zeros((len(X),6))
            for i in self.tree_dictionary:
                a=[]
                regions=self.tree_dictionary[i].apply(X)
                for j in regions:
                    gamma = self.tree_gamma_dictionary[i][j]
                    a.append(gamma)
                b=np.add(a,b)
            return b
                
    def predict(self,X):
        F_val = self.F(X)
        predictions = self.Pk(F_val)
        output = []
        for i in predictions:
            temp =[]
            label = np.argmax(i)
            for j in range(len(i)):
                if j==label:
                    temp.append(1)
                else:
                    temp.append(0)      
            output.append(temp)
        return output
    
    def Pk(self,F_val):
        output=[]
        for row in F_val:
            b=np.sum(np.exp(row))
            a=np.exp(row)
            d = a/b 
            output.append(d)
        return output
    
    def fit(self,X,Y):
        
        for i in range(self.n_tree):
            numerator_dict ={}
            denominator_dict={}
            F_val = self.F(X)
            predictions = self.Pk(F_val)
            residuals = np.asarray(np.array(Y)-predictions)
            regressor = DecisionTreeRegressor(random_state=0,max_depth=self.depth)
            regressor.fit(X,residuals)

            X_regions = regressor.apply(X)
            
            for j in range(len(X_regions)):
                if X_regions[j] not in numerator_dict:
                    numerator_dict[X_regions[j]]=residuals[j]
                    denominator_dict[X_regions[j]]=abs(residuals[j])*(1-abs(residuals[j]))
                else:
                    numerator_dict[X_regions[j]]+=residuals[j]
                    denominator_dict[X_regions[j]]+=abs(residuals[j])*(1-abs(residuals[j]))
                    
            temp_dict={}
            for k in numerator_dict:
                temp_dict[k]=(numerator_dict[k]/(6*denominator_dict[k]))
            
            self.tree_gamma_dictionary[i] = temp_dict
            self.tree_dictionary[i]=regressor